In [1]:
import os
path = "/Users/patricia/Documents/code/python-code/behavior-detection/src"
os.chdir(path)

from core.preprocessors.data_loader import DataLoader

# Load data
file_path = '../data/new_logs_labels.csv'
df = DataLoader.load_data(file_path, delimiter=';')

In [2]:
df.head()

,id_log,aluno,grupo,num_dia,num_log,log_type,ultimo_passo_correto,verificado_com_mouse,verificado_com_teclado,idle_time_acumulado,...,comportamento_off_task,comportamento_on_system,comportamento_indefinido,ultimo_comportamento,ultimo_comportamento_on_task,ultimo_comportamento_on_task_conversation,ultimo_comportamento_on_task_out,ultimo_comportamento_off_task,ultimo_comportamento_on_system,ultimo_comportamento_indefinido
0,8224,1,2,1,1,step_verification,0,0,1,0,...,0,0,0,?,0,0,0,0,0,1
1,527786,1,2,1,2,user_idle,1,0,0,2,...,0,0,0,ON TASK,1,0,0,0,0,0
2,527787,1,2,1,3,user_idle,0,0,0,4,...,0,0,0,ON TASK,1,0,0,0,0,0
3,527788,1,2,1,4,user_idle,0,0,0,6,...,0,0,0,ON TASK,1,0,0,0,0,0
4,527789,1,2,1,5,user_idle,0,0,0,8,...,0,0,0,ON TASK,1,0,0,0,0,0


In [3]:
!pip install ydata-profiling

In [4]:
def analyze_dataframe(df):
    from ydata_profiling import ProfileReport

    profile = ProfileReport(df, title="Dataset Analysis Report",
                            minimal=False,
                            correlations={
                                "pearson": {"calculate": True},
                                "spearman": {"calculate": True},
                                "kendall": {"calculate": True},
                                "phi_k": {"calculate": True},
                            })
    return profile


# Uso
profile = analyze_dataframe(df)
file_path = '../../output/dataset_report.html'
profile.to_file(file_path)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/opt/anaconda3/envs/projetos_ML/lib/python3.11/site-packages/ydata_profiling/model/pandas/duplicates_pandas.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  .reset_index(name=duplicates_key)
/opt/anaconda3/envs/projetos_ML/lib/python3.11/site-packages/ydata_profiling/model/pandas/duplicates_pandas.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  .reset_index(name=duplicates_key)
/opt/anaconda3/envs/projetos_ML/lib/python3.11/site-packages/ydata_profiling/model/pandas/duplicates_pandas.py:40: PerformanceWarning: DataFrame

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
from core.exploration.data_exploration import DataExploration

# Criar arquivo de metadados
DataExploration.create_metadata_file(df, '../data/metadata.csv')

Metadados salvos em ../data/metadata.csv


In [6]:
# Verificar valores únicos e a quantidade de cada um nas colunas especificadas
colunas = ['grupo', 'aluno', 'num_dia']
for coluna in colunas:
    print(f"Valores únicos e contagem para a coluna '{coluna}':")
    print(df[coluna].nunique())
    print(df[coluna].unique())
    print("\n")

print(df.groupby(['grupo', 'aluno']).ngroups)
print(df.groupby(['grupo', 'aluno']).size())

Valores únicos e contagem para a coluna 'grupo':
10
[ 2  5  3  1  7  6  9  8 10  4]


Valores únicos e contagem para a coluna 'aluno':
30
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30]


Valores únicos e contagem para a coluna 'num_dia':
10
[ 1  2  3  4  5  6  7  8  9 10]


30
grupo  aluno
1      4        173
       14       185
       27       195
2      1        184
       20       192
       25       149
3      3        191
       11       283
       22       177
4      10       189
       12       186
       29       161
5      2        188
       18       188
       24       159
6      6        167
       19       158
       21       185
7      5        181
       15       145
       30       199
8      8        164
       17       183
       23       214
9      7        202
       13       186
       26       166
10     9        188
       16       205
       28       182
dtype: int64


In [7]:
def listar_features_constantes(df):
    # Encontrar colunas onde todos os valores são iguais
    features_constantes = [col for col in df.columns if df[col].nunique() == 1]

    # Imprimir cada feature constante e seu valor
    print(
        f"Features com valor constante ({len(features_constantes)} features):")
    for feature in sorted(features_constantes):
        # pegamos o primeiro valor já que são todos iguais
        valor = df[feature].iloc[0]
        print(f"- {feature} (valor constante: {valor})")

    return features_constantes

listar_features_constantes(df)

Features com valor constante (59 features):
- cat_dificil_4 (valor constante: 0)
- cat_facil_2 (valor constante: 0)
- click_atalhos (valor constante: 0)
- click_bugs (valor constante: 0)
- click_indefinidos (valor constante: 0)
- eq_AF_erro_clip (valor constante: 0)
- eq_AF_erro_diario (valor constante: 0)
- eq_AF_erro_medio_diario_turma (valor constante: 0)
- eq_AF_erro_medio_total_turma (valor constante: 0)
- eq_AF_erro_total (valor constante: 0)
- eq_DV (valor constante: 0)
- eq_DV_acerto_clip (valor constante: 0)
- eq_DV_acerto_diario (valor constante: 0)
- eq_DV_acerto_medio_diario_turma (valor constante: 0)
- eq_DV_acerto_medio_total_turma (valor constante: 0)
- eq_DV_acerto_total (valor constante: 0)
- eq_DV_fator_clip (valor constante: 0)
- eq_DV_fator_diario (valor constante: 0)
- eq_DV_fator_medio_diario_turma (valor constante: 0)
- eq_DV_fator_medio_total_turma (valor constante: 0)
- eq_DV_fator_total (valor constante: 0)
- eq_MF_erro_clip (valor constante: 0)
- eq_MF_erro_d

['type_window_resize',
 'type_start_out_of_sinc_server_time',
 'type_user_log_out',
 'click_atalhos',
 'click_bugs',
 'click_indefinidos',
 'cat_facil_2',
 'cat_dificil_4',
 'eq_DV',
 'eq_DV_acerto_clip',
 'eq_DV_acerto_diario',
 'eq_DV_acerto_total',
 'eq_DV_acerto_medio_diario_turma',
 'eq_DV_acerto_medio_total_turma',
 'eq_AF_erro_clip',
 'eq_MF_erro_clip',
 'eq_MM_erro_clip',
 'eq_AF_erro_diario',
 'eq_MF_erro_diario',
 'eq_AF_erro_total',
 'eq_MF_erro_total',
 'eq_AF_erro_medio_diario_turma',
 'eq_MF_erro_medio_diario_turma',
 'eq_AF_erro_medio_total_turma',
 'eq_MF_erro_medio_total_turma',
 'eq_DV_fator_clip',
 'eq_DV_fator_diario',
 'eq_DV_fator_total',
 'eq_DV_fator_medio_diario_turma',
 'eq_DV_fator_medio_total_turma',
 'misc_EqPrim_Ad_Inc_clip',
 'misc_OI_Dv_Plus_Dv_Minus_clip',
 'misc_OI_Mt_Minus_Ad_clip',
 'misc_OI_Mt_Plus_Mt_Plus_clip',
 'misc_OI_Dv_Plus_Dv_Plus_clip',
 'misc_OI_Mt_Plus_Dv_Minus_clip',
 'misc_OI_Mt_Plus_Ad_clip',
 'misc_OI_Dv_Minus_Dv_Plus_clip',
 'misc_OI